In [93]:
import pandas as pd
import glob
from google.cloud import storage
from google.cloud import bigquery

In [87]:
def extract():
        
    cf=glob.glob("csv/*.csv")
    csv=pd.concat([pd.read_csv(f) for f in cf], ignore_index=True)

    xf=glob.glob("excel/*.xlsx")
    exc=pd.concat([pd.read_excel(f) for f in xf], ignore_index=True)

    jf=glob.glob("json/*.json")
    js=pd.concat([pd.read_json(f, encoding="latin1", lines=True) for f in jf], ignore_index=True)


    new_df_exc=exc.copy()
    new_df_exc=new_df_exc.drop(columns=["Discount(%)","FinalTotal"])   #selecting needed columns


    all_join_df=pd.concat([js, csv, new_df_exc],ignore_index=True)   # joining all datasource
    all_join_df=all_join_df.drop_duplicates(subset=["OrderID"])   #removing duplicates using the column orderid
    all_join_df=all_join_df.reset_index(drop=True)

    print(cf,jf,xf)

    return all_join_df









In [89]:
extract()

['csv/sales_dataset.csv'] ['json/sales.json', 'json/sales_dataset.json'] ['excel/sales_dataset.xlsx']


,OrderID,CustomerID,Product,Category,Quantity,UnitPrice,TotalPrice,OrderDate,Region
0,ORDJ3000,CUST0164,Phone,Electronics,4,1714.24,6856.96,2022-04-17,North
1,ORDJ3001,CUST0190,Monitor,Electronics,5,1098.52,5492.60,2022-05-02,South
2,ORDJ3002,CUST0058,Tablet,Electronics,3,233.96,701.88,2023-04-04,North
3,ORDJ3003,CUST0174,Phone,Electronics,5,1105.94,5529.70,2023-10-26,West
4,ORDJ3004,CUST0009,Laptop,Electronics,3,95.82,287.46,2022-12-10,North
...,...,...,...,...,...,...,...,...,...
4995,ORDX4995,CUST0133,Headphones,Accessories,8,1355.39,10843.12,2024-09-03,East
4996,ORDX4996,CUST0138,Headphones,Accessories,1,378.50,378.50,2023-03-20,South
4997,ORDX4997,CUST0138,Mouse,Accessories,4,558.25,2233.00,2022-02-03,South
4998,ORDX4998,CUST0075,Tablet,Electronics,3,1681.95,5045.85,2023-12-17,East


In [ ]:

def load(data_to_load, gcp_bucket_name, gcs_object, project_id, dataset_id, table_id):
    if data_to_load is None:
        print("Data Unavailable to load to cloud")
        return
    try:
        print(f"Uploading to Google Bucket '{gcp_bucket_name}' at '{gcs_object}'")
        client=storage.Client()
        bucket= client.bucket(gcp_bucket_name)
        blob=bucket.blob(gcs_object)
        dt=pd.DataFrame(data_to_load)
        csv_data=dt.to_csv(index=False)
        blob.upload_from_string(csv_data, content_type='text/csv')

        #Verify Upload
        if blob.exists():
            gcs_url=f"gs://{gcp_bucket_name}/{gcs_object}"
            print("Uploaded successfully to :",gcs_url)

        else:
            raise Exception("Uploading verification failed")
        

        #Loading to Bigquery from GCS
        print(f"Loading to Bigquery fromGCS ",gcs_url)
        bg=bigquery.Client(project=project_id)

        #Verify if dataset(table) exists 
        id=bg.dataset(id)
        if not id.exists():
            dataset=bigquery.Dataset(id)
            dataset=bg.create_dataset(id)
            print("Dataset Created Successfully")


            table_ref=id.table(table_id)
            job_config=bigquery.LoadJobConfig(
                source_format=bigquery.SourceFormat.CSV,
                skip_leading_rows=1,  # Skip header
                write_disposition="WRITE_TRUNCATE",  # Replace table if exists
                autodetect=True,  # Auto-detect schema
            )

            load_job = bg.load_table_from_uri(
            [gcs_url], table_ref, job_config=job_config
            )
            load_job.result()  # Wait for job to complete

            # Post-load validation
        table = bg.get_table(table_ref)
        expected_rows = len(data_to_load)
        if table.num_rows == expected_rows:
            print(f"Load successful! Loaded {table.num_rows} rows to '{project_id}.{dataset_id}.{table_id}'.")
        else:
            print(f"Warning: Loaded {table.num_rows} rows, expected {expected_rows}.")
            
    except Exception as e:
        print(f"Error during loading: {e}")
        if 'bigquery' in str(type(e)).lower():
            print("Tip: Check BigQuery permissions and dataset existence.")
        if 'storage' in str(type(e)).lower():
            print("Tip: Check GCS bucket name and permissions.")







